## libraries used throughout the program

In [1]:
import re
import os
from subprocess import call
import json
import pandas as pd
import math
import sys

## this method is used to split a paragraph into sentences
- this creates one entry per sentence in the json file
- it is also used then to extract one sentence from the original mp3 file

In [2]:
def splitParagraphIntoSentences(paragraph):
    # preprocess the paragraph first before splitting into sentences
    paragraph = re.sub(r'U\.S\.', 'US', paragraph)
    paragraph = re.sub(r'Mr\.', 'Mr', paragraph)
    paragraph = re.sub(r'Mrs\.', 'Mrs', paragraph)
    paragraph = re.sub(r'Ms\.', 'Ms', paragraph)
    paragraph = re.sub(r'Operator:', 'O:', paragraph)
    paragraph = re.sub(r'\[ph\]', '', paragraph)
    paragraph = re.sub(r'\(\d+:\d+\)', '', paragraph)
    
#     if the paragraph starts with a Question or Answer tag (Q&A section), temporary store the name ...
#     QnA = re.findall('\<.*\>:\s', paragraph)

    # replace the question and answer tag with Q: and A: respectively
    paragraph = re.sub(r'\<Q.*\>:\s', 'Q: ', paragraph)
    paragraph = re.sub(r'\<A.*\>:\s', 'A: ', paragraph)
    # split the paragraph after every period what is followed by a white space
    sentenceEnders = re.compile('(?<=[\.\!\?])\s')
    sentenceList = sentenceEnders.split(paragraph)
    
#     if len(QnA):
#         add the name later again to restore previous structure
#         sentenceList[0] = QnA[0] + sentenceList[0]

    return sentenceList

## here the text gets preprocessed the first time and stored in a new file in a subfolder
- the original file has a new line after every 11th word
- with preprocessing these newlines are being removed
- this means that one whole conversation of one person gets stored as one paragraph in the new file
## stop the execution after here to make manual adjustments to the text

In [3]:
def processTextFileToParagraph(path, text_file_name, process_folder, text_file):
    # open the text file for the current mp3 file
    with open(path + text_file_name + '.txt') as f:
        # create a new file for the preprocessed text
        with open(process_folder + text_file + 'PreprocessToParagraphs.txt', "w") as f1:
            # while reading the first file write in the second
            for line in f:
                line.replace(".", ". ")
                # only remove newline characters if the line is not empty
                if line not in ['\n', '\r\n']:
                    f1.write(line.rstrip('\n'))
                else:
                    f1.write(line)

## here the text gets processed a second time and stored again in a new file in the same folder
- this step is splitting the paragraph back to sentences
- the json file will include then one sentence per entry

In [4]:
def processTextFileToSentences(process_folder, text_file):
    # open the just created file with processed text
    with open(process_folder + text_file + "PreprocessToParagraphs.txt") as f:
        # create another file for the second preprocessing step
        with open(process_folder + text_file + 'PreprocessToSentences.txt', "w") as f1:
            for line in f:
                not_needed = ['MANAGEMENT DISCUSSION SECTION',
                             'Wire: Bloomberg Transcripts',
                             'CallSource: ',
                             'For more event information and transcripts',
                             'Q&A',
                             'This transcript may not be 100 percent',
                             'Copyright (c)',
                             'END OF STORY']
                if not any(x in line for x in not_needed) or ' Q&A' in line:
                    # split paragraph into sentences
                    sentences = splitParagraphIntoSentences(line)
                    for s in sentences:
                        if s is not '':
                            # remove double white spaces from the text
                            f1.write(' '.join(s.split()) + '\n')

## This method creates the json file for the earlier specified original voice mp3
#### with the help of aeneas the mp3 gets analyzed and aligned with the preprocessed sentences from the file

In [5]:
def createJSON(path, file, process_folder, text_file):
    # call the aeneas library and create the json file
    call(["python",
          "-m", 
          "aeneas.tools.execute_task",
          path + "/" + file, # original mp3 file
          process_folder + text_file + "PreprocessToSentences.txt", # sentences for the mp3 file
          "task_language=eng|os_task_file_format=json|is_text_type=plain",
          process_folder + text_file + ".json"]) # json file containing the sentences and begin and end time

## This method loops through the json file again and extract the begin and end time of each sentence
- round to 2 decimals
## the filename contains the original filename and the created id in the json file to keep the files apart and prevent overwriting
- also a maximum difference between begin and end time of each sentence gets calculated for future purposes

- this script runs for quite a long time creating small mp3 files for each sentence

In [6]:
def createMP3withJSON(path, file, mp3_folder, process_folder, text_file):
    # open the json file
    with open(process_folder + text_file + ".json") as json_file:
        json_object = json.load(json_file)
        # transform the json file to a dataframe using pandas
        df = pd.DataFrame.from_dict(json_object, orient='columns')
        maxDiff = 0
        # for each sentence
        for sentence in df['fragments']:
            # calculate difference between begin and end time and round to 2 decimals
            diff = math.ceil((float(sentence['end']) - float(sentence['begin'])) * 100) / 100
            # update max difference
            if diff > maxDiff:
                maxDiff = diff
            # skip entries with 0 seconds of sound
            if sentence['lines'][0]:
                sys.stdout.write("\r" + file + " File ID: " + sentence['id'])
                sys.stdout.flush()
                # create mp3 file for sentence
                call(["ffmpeg",
                  "-ss",
                  str(sentence['begin']), # begin time
                  "-t",
                  str(diff), # difference to end time
                  "-i",
                  path + "/" + file, # original mp3 file
                  mp3_folder + text_file + sentence['id'] + ".mp3"]) # mp3 file for sentence
        print(" - Done")
        

# start the process 
## extract the mp3 file names from the defined path
- the path containing all the mp3 files
- get the filename from the path
- path contains the path to the audio files
- file contains the name of file
- replace_strings contains everything what the filename of the audio contains, but not the filename of the respective text file
- the 2 quotes before and after the text_file are to be used in case the filename of the text file contains something the audio file does not

In [7]:
# path to the original mp3 file
path = 'btq113/'
for files in os.listdir(path):
    if files.endswith(".mp3"): 
        # get the complete path to the file
        file = os.path.basename(files)
        # by default remove white spaces from the text file name
        text_file = (os.path.splitext(file)[0]).lower().replace(" ", "")

        # if there is something what needs to be removed from the name
        # remember to write things lower case here
        replace_strings = ["nokia","-","20"] 
        for replace_string in replace_strings:
            if replace_string:
                # only remove other things if stated
                text_file = text_file.replace(replace_string, "")
        # add aditional information in case something got replaced
        text_file = "" + text_file + ""

        # create a subfolder for the original mp3 file containing the filename if the folder does not yet exist
        process_folder = path + text_file + '/'
        mp3_folder = process_folder + "mp3/"
        if not os.path.exists(process_folder):
            os.makedirs(process_folder) 
        if not os.path.exists(mp3_folder):
            os.makedirs(mp3_folder)

        # add possible extra text for the filename otherwise leave it empty
        text_file_name = "" + text_file + ""
        
        # call the preprocessing for each mp3 file
        processTextFileToParagraph(path, text_file_name, process_folder, text_file)
        processTextFileToSentences(process_folder, text_file)
        
        # create the json for this mp3 file
        createJSON(path, file, process_folder, text_file)
        
        # split the mp3 into sentences
        createMP3withJSON(path, file, mp3_folder, process_folder, text_file)
        

BT q1 13cut.mp3 File ID: f000287 - Done
BT q1 13cutext9.mp3 File ID: f000033 - Done
BT q1 13cutext8.mp3 File ID: f000026 - Done
BT q1 13cutext13.mp3 File ID: f000036 - Done
BT q1 13cutext12.mp3 File ID: f000047 - Done
BT q1 13cutext10.mp3 File ID: f000046 - Done
BT q1 13cutext11.mp3 File ID: f000072 - Done
BT q1 13cutext15.mp3 File ID: f000036 - Done
BT q1 13cutext14.mp3 File ID: f000045 - Done
BT q1 13cutext16.mp3 File ID: f000057 - Done
BT q1 13cutext3.mp3 File ID: f000036 - Done
BT q1 13cutext2.mp3 File ID: f000036 - Done
BT q1 13cutext1.mp3 File ID: f000033 - Done
BT q1 13cutext5.mp3 File ID: f000026 - Done
BT q1 13cutext4.mp3 File ID: f000029 - Done
BT q1 13cutext6.mp3 File ID: f000018 - Done
BT q1 13cutext7.mp3 File ID: f000049 - Done
